# API data load

In [63]:
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus
from urllib.parse import unquote
import requests
import xml.etree.ElementTree as ET
import pandas as pd

In [64]:
KEY = 'M%2B4%2FqUiadT8X8PhgFjaQLDu%2BIOgPMURfGsOX%2FmVxwHQVJgnVR%2FMPjDYXkuQNwUFbZXlfnX5Lls3SUCiCLIFjgQ%3D%3D'
API_KEY = requests.utils.unquote(KEY)
API_KEY = unquote(KEY)

In [65]:
routeId = [
    '234000013',
    234000130,
    204000056,
    204000046,
    204000041,
    204000070,
    204000059,
    204000060,
    204000022,
    204000039,
    204000040,
    204000067,
    204000044,
    204000147
]

In [66]:
# getBusPosByRtid
# 노선ID로 차량들의 위치정보를 조회한다
# 서울
url_seoul_location = 'http://ws.bus.go.kr/api/rest/buspos/getBusPosByRtid'
queryParams_seoul_location = '?' + urlencode({ quote_plus('ServiceKey') : API_KEY, quote_plus('busRouteId') : '0' })

In [ ]:
# 버스도착정보조회 서비스
url_gyeonggi_arrive = 'http://openapi.gbis.go.kr/ws/rest/busarrivalservice'
queryParams_gyeonggi_arrive = '?' + urlencode({ quote_plus('ServiceKey') : API_KEY, quote_plus('busRouteId') : '204000046' })

In [81]:
# 경유정류소목록조회 buslocationservice
# 해당 노선이 정차하는 경유정류소 목록과 정류소명, 중앙차로여부, 회차점, 좌표값 등을 제공한다.
url_gyeonggi_location = 'http://openapi.gbis.go.kr/ws/rest/buslocationservice'
for x in routeId :
    print(x)
    queryParams_gyeonggi_location = '?' + urlencode({ quote_plus('serviceKey') : API_KEY, quote_plus('routeId') : x })
    ##for문 실험
    buslocation_gyeonggi = url_gyeonggi_location + queryParams_gyeonggi_location
    location_gyeonggi = Request(buslocation_gyeonggi)
    location_gyeonggi.get_method = lambda: 'GET'
    location_gyeonggi = urlopen(location_gyeonggi).read().decode('utf8')
    
    xtree = ET.fromstring(location_gyeonggi)

    rows = []

    resultCode = int(xtree[1].find("resultCode").text)
    queryTime = xtree[1].find("queryTime").text

    if resultCode == 0 :
        for node in xtree[2] :
            n_endBus = node.find("endBus").text
            n_lowPlate = node.find("lowPlate").text
            n_plateNo = node.find("plateNo").text
            n_plateType = node.find("plateType").text
            n_remainSeatCnt = node.find("remainSeatCnt").text
            n_routeId = node.find("routeId").text
            n_stationId = node.find("stationId").text
            n_stationSeq = node.find("stationSeq").text
        
            rows.append({"endBus":n_endBus,
                     "lowPlate":n_lowPlate,
                     "plateNo":n_plateNo,
                     "plateType":n_plateType,
                     "remainSeatCnt":n_remainSeatCnt,
                     "routeId":n_routeId,
                     "stationId":n_stationId,
                     "stationSeq":n_stationSeq})
    else :
        print("There is no bus\n")

    rows.append({"queryTime":queryTime})
    rows.append({"resultCode":resultCode})

    for i in rows :
        print(i)
        print('\n')

234000013
{'endBus': '0', 'lowPlate': '0', 'plateNo': '경기77바2016', 'plateType': '3', 'remainSeatCnt': '39', 'routeId': '234000013', 'stationId': '123000083', 'stationSeq': '49'}


{'endBus': '0', 'lowPlate': '0', 'plateNo': '경기77바3443', 'plateType': '3', 'remainSeatCnt': '15', 'routeId': '234000013', 'stationId': '206000548', 'stationSeq': '72'}


{'endBus': '0', 'lowPlate': '0', 'plateNo': '경기77바2040', 'plateType': '3', 'remainSeatCnt': '45', 'routeId': '234000013', 'stationId': '206000654', 'stationSeq': '27'}


{'queryTime': '2020-11-10 20:35:02.117'}


{'resultCode': 0}


234000130
{'endBus': '0', 'lowPlate': '0', 'plateNo': '경기77바1282', 'plateType': '3', 'remainSeatCnt': '43', 'routeId': '234000130', 'stationId': '277102684', 'stationSeq': '13'}


{'endBus': '0', 'lowPlate': '0', 'plateNo': '경기77바1213', 'plateType': '3', 'remainSeatCnt': '44', 'routeId': '234000130', 'stationId': '277101770', 'stationSeq': '78'}


{'endBus': '0', 'lowPlate': '0', 'plateNo': '경기77바1179', 'plateType

{'endBus': '0', 'lowPlate': '0', 'plateNo': '경기70아1706', 'plateType': '3', 'remainSeatCnt': '-1', 'routeId': '204000022', 'stationId': '206000016', 'stationSeq': '50'}


{'endBus': '0', 'lowPlate': '0', 'plateNo': '경기70아1787', 'plateType': '3', 'remainSeatCnt': '-1', 'routeId': '204000022', 'stationId': '204000085', 'stationSeq': '95'}


{'endBus': '0', 'lowPlate': '1', 'plateNo': '경기70아1963', 'plateType': '3', 'remainSeatCnt': '-1', 'routeId': '204000022', 'stationId': '205000064', 'stationSeq': '108'}


{'endBus': '0', 'lowPlate': '0', 'plateNo': '경기70아1568', 'plateType': '3', 'remainSeatCnt': '-1', 'routeId': '204000022', 'stationId': '206000157', 'stationSeq': '29'}


{'endBus': '0', 'lowPlate': '0', 'plateNo': '경기70아1779', 'plateType': '3', 'remainSeatCnt': '-1', 'routeId': '204000022', 'stationId': '206000556', 'stationSeq': '51'}


{'endBus': '0', 'lowPlate': '0', 'plateNo': '경기70아1710', 'plateType': '3', 'remainSeatCnt': '-1', 'routeId': '204000022', 'stationId': '206000300', '

{'endBus': '0', 'lowPlate': '0', 'plateNo': '경기70아1548', 'plateType': '3', 'remainSeatCnt': '-1', 'routeId': '204000040', 'stationId': '204000132', 'stationSeq': '14'}


{'endBus': '0', 'lowPlate': '0', 'plateNo': '경기70아1560', 'plateType': '3', 'remainSeatCnt': '-1', 'routeId': '204000040', 'stationId': '277102644', 'stationSeq': '69'}


{'endBus': '0', 'lowPlate': '0', 'plateNo': '경기70아1552', 'plateType': '3', 'remainSeatCnt': '-1', 'routeId': '204000040', 'stationId': '205000094', 'stationSeq': '95'}


{'endBus': '0', 'lowPlate': '0', 'plateNo': '경기70아1562', 'plateType': '3', 'remainSeatCnt': '-1', 'routeId': '204000040', 'stationId': '206000795', 'stationSeq': '60'}


{'endBus': '0', 'lowPlate': '0', 'plateNo': '경기70아1551', 'plateType': '3', 'remainSeatCnt': '-1', 'routeId': '204000040', 'stationId': '205000094', 'stationSeq': '95'}


{'endBus': '0', 'lowPlate': '0', 'plateNo': '경기70아1559', 'plateType': '3', 'remainSeatCnt': '-1', 'routeId': '204000040', 'stationId': '205000002', 's

In [82]:
url_gyeonggi_location = 'http://openapi.gbis.go.kr/ws/rest/buslocationservice'
for x in routeId :
    print(x)
    queryParams_gyeonggi_location = '?' + urlencode({ quote_plus('serviceKey') : API_KEY, quote_plus('routeId') : x })
    ##for문 실험
    buslocation_gyeonggi = url_gyeonggi_location + queryParams_gyeonggi_location
    location_gyeonggi = Request(buslocation_gyeonggi)
    location_gyeonggi.get_method = lambda: 'GET'
    location_gyeonggi = urlopen(location_gyeonggi).read().decode('utf8')
    print(location_gyeonggi)

234000013
<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><comMsgHeader><errMsg>NORMAL SERVICE.</errMsg><returnCode>00</returnCode></comMsgHeader><msgHeader><queryTime>2020-11-10 20:35:25.570</queryTime><resultCode>0</resultCode><resultMessage>정상적으로 처리되었습니다.</resultMessage></msgHeader><msgBody><busLocationList><endBus>0</endBus><lowPlate>0</lowPlate><plateNo>경기77바3443</plateNo><plateType>3</plateType><remainSeatCnt>15</remainSeatCnt><routeId>234000013</routeId><stationId>206000548</stationId><stationSeq>72</stationSeq></busLocationList><busLocationList><endBus>0</endBus><lowPlate>0</lowPlate><plateNo>경기77바2016</plateNo><plateType>3</plateType><remainSeatCnt>39</remainSeatCnt><routeId>234000013</routeId><stationId>123000083</stationId><stationSeq>49</stationSeq></busLocationList><busLocationList><endBus>0</endBus><lowPlate>0</lowPlate><plateNo>경기77바2040</plateNo><plateType>3</plateType><remainSeatCnt>45</remainSeatCnt><routeId>234000013</routeId><stationId>206000552</st

<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><comMsgHeader><errMsg>NORMAL SERVICE.</errMsg><returnCode>00</returnCode></comMsgHeader><msgHeader><queryTime>2020-11-10 20:35:29.148</queryTime><resultCode>0</resultCode><resultMessage>정상적으로 처리되었습니다.</resultMessage></msgHeader><msgBody><busLocationList><endBus>0</endBus><lowPlate>0</lowPlate><plateNo>경기70아8795</plateNo><plateType>3</plateType><remainSeatCnt>24</remainSeatCnt><routeId>204000070</routeId><stationId>121900201</stationId><stationSeq>30</stationSeq></busLocationList><busLocationList><endBus>0</endBus><lowPlate>0</lowPlate><plateNo>경기70아7332</plateNo><plateType>3</plateType><remainSeatCnt>44</remainSeatCnt><routeId>204000070</routeId><stationId>206000606</stationId><stationSeq>61</stationSeq></busLocationList><busLocationList><endBus>0</endBus><lowPlate>0</lowPlate><plateNo>경기70아6140</plateNo><plateType>3</plateType><remainSeatCnt>0</remainSeatCnt><routeId>204000070</routeId><stationId>277103679</stationId><st

<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><comMsgHeader><errMsg>NORMAL SERVICE.</errMsg><returnCode>00</returnCode></comMsgHeader><msgHeader><queryTime>2020-11-10 20:35:37.367</queryTime><resultCode>0</resultCode><resultMessage>정상적으로 처리되었습니다.</resultMessage></msgHeader><msgBody><busLocationList><endBus>0</endBus><lowPlate>0</lowPlate><plateNo>경기70아1761</plateNo><plateType>3</plateType><remainSeatCnt>-1</remainSeatCnt><routeId>204000039</routeId><stationId>204000204</stationId><stationSeq>54</stationSeq></busLocationList><busLocationList><endBus>0</endBus><lowPlate>0</lowPlate><plateNo>경기70아1586</plateNo><plateType>3</plateType><remainSeatCnt>-1</remainSeatCnt><routeId>204000039</routeId><stationId>204000086</stationId><stationSeq>63</stationSeq></busLocationList><busLocationList><endBus>0</endBus><lowPlate>0</lowPlate><plateNo>경기70아1621</plateNo><plateType>3</plateType><remainSeatCnt>-1</remainSeatCnt><routeId>204000039</routeId><stationId>204000088</stationId><s

<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><comMsgHeader><errMsg>NORMAL SERVICE.</errMsg><returnCode>00</returnCode></comMsgHeader><msgHeader><queryTime>2020-11-10 20:35:41.023</queryTime><resultCode>0</resultCode><resultMessage>정상적으로 처리되었습니다.</resultMessage></msgHeader><msgBody><busLocationList><endBus>0</endBus><lowPlate>0</lowPlate><plateNo>경기70아1855</plateNo><plateType>3</plateType><remainSeatCnt>-1</remainSeatCnt><routeId>204000044</routeId><stationId>205000059</stationId><stationSeq>87</stationSeq></busLocationList><busLocationList><endBus>0</endBus><lowPlate>0</lowPlate><plateNo>경기70아1877</plateNo><plateType>3</plateType><remainSeatCnt>-1</remainSeatCnt><routeId>204000044</routeId><stationId>205000368</stationId><stationSeq>18</stationSeq></busLocationList><busLocationList><endBus>0</endBus><lowPlate>0</lowPlate><plateNo>경기70아1813</plateNo><plateType>3</plateType><remainSeatCnt>-1</remainSeatCnt><routeId>204000044</routeId><stationId>205000013</stationId><s

In [78]:
buslocation_seoul = url_seoul_location + queryParams_seoul_location
busarrive_gyeonggi = url_gyeonggi_arrive + queryParams_gyeonggi_arrive
buslocation_gyeonggi = url_gyeonggi_location + queryParams_gyeonggi_location

location_seoul = Request(buslocation_seoul)
location_seoul.get_method = lambda: 'GET'
location_seoul = urlopen(location_seoul).read().decode('utf8')

location_gyeonggi = Request(buslocation_gyeonggi)
location_gyeonggi.get_method = lambda: 'GET'
location_gyeonggi = urlopen(location_gyeonggi).read().decode('utf8')

arrive_gyeonggi = Request(busarrive_gyeonggi)
arrive_gyeonggi.get_method = lambda: 'GET'
arrive_gyeonggi = urlopen(arrive_gyeonggi).read().decode('utf8')

In [73]:
url_gyeonggi_location = 'http://openapi.gbis.go.kr/ws/rest/buslocationservice'
queryParams_gyeonggi_location = '?' + urlencode({ quote_plus('serviceKey') : API_KEY, quote_plus('routeId') : '204000046' })

buslocation_gyeonggi = url_gyeonggi_location + queryParams_gyeonggi_location

location_gyeonggi = Request(buslocation_gyeonggi)
location_gyeonggi.get_method = lambda: 'GET'
location_gyeonggi = urlopen(location_gyeonggi).read().decode('utf8')

print(location_gyeonggi)
xtree = ET.fromstring(location_gyeonggi)

rows = []

resultCode = int(xtree[1].find("resultCode").text)
queryTime = xtree[1].find("queryTime").text

print(resultCode)
if resultCode == 0 :
    for node in xtree[2] :
        n_endBus = node.find("endBus").text
        n_lowPlate = node.find("lowPlate").text
        n_plateNo = node.find("plateNo").text
        n_plateType = node.find("plateType").text
        n_remainSeatCnt = node.find("remainSeatCnt").text
        n_routeId = node.find("routeId").text
        n_stationId = node.find("stationId").text
        n_stationSeq = node.find("stationSeq").text
        
        rows.append({"endBus":n_endBus,
                     "lowPlate":n_lowPlate,
                     "plateNo":n_plateNo,
                     "plateType":n_plateType,
                     "remainSeatCnt":n_remainSeatCnt,
                     "routeId":n_routeId,
                     "stationId":n_stationId,
                     "stationSeq":n_stationSeq})
else :
    print("There is no bus\n")

rows.append({"queryTime":queryTime})
rows.append({"resultCode":resultCode})

for x in rows :
    print(x)
    print('\n')

<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><comMsgHeader><errMsg>NORMAL SERVICE.</errMsg><returnCode>00</returnCode></comMsgHeader><msgHeader><queryTime>2020-11-10 20:28:52.682</queryTime><resultCode>0</resultCode><resultMessage>정상적으로 처리되었습니다.</resultMessage></msgHeader><msgBody><busLocationList><endBus>0</endBus><lowPlate>0</lowPlate><plateNo>경기77바1097</plateNo><plateType>3</plateType><remainSeatCnt>21</remainSeatCnt><routeId>204000046</routeId><stationId>277103392</stationId><stationSeq>37</stationSeq></busLocationList><busLocationList><endBus>0</endBus><lowPlate>0</lowPlate><plateNo>경기70아7361</plateNo><plateType>3</plateType><remainSeatCnt>41</remainSeatCnt><routeId>204000046</routeId><stationId>101000005</stationId><stationSeq>27</stationSeq></busLocationList><busLocationList><endBus>0</endBus><lowPlate>0</lowPlate><plateNo>경기70아8601</plateNo><plateType>3</plateType><remainSeatCnt>39</remainSeatCnt><routeId>204000046</routeId><stationId>277104177</stationId><s